# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования** - ответить на четыре вопроса:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Структура исследования:
1. Изучение информации представленного датасета (датасет со статистикой о платежеспособности клиентов банка).
2. Предобработка данных.
3. Ответы на поставленные вопросы.


Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Изучение информации в исходном датасете

In [1]:
# открытие файла с локальной машины, в случае обработки в jupyterhub - чтение по указанному заказчиком пути
# вывод первых 10 строк для предварительной оценки табличных данных

import pandas as pd
try:
    data = pd.read_csv('data.csv')
    display(data.head(10))
except:
    data = pd.read_csv('/datasets/data.csv')
    display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Рассмотрим общую информацию о таблице:

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


___
В исходном датасете, состоящем из 21 525 строк, представлена статистика о платежеспособности клиентов.

Согласно документации:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита


Так как гипотезы исследования обозначают взаимосвязь возврата кредита в срок с различными характеристиками кредиторов, то основой для сопоставления будет служить столбец **debt**. Названия столбцов соответствуют правилам стиля. 

В столбцах **days_employed** и **total_income** меньше значений, значит имеются пропуски. Также по первым строкам таблицы, видим отрицательные значения в стаже клиентов, что является ошибкой в данных

Приступим к изучению данных по столбцам и выявим задачи предобработки.

In [3]:
# процент отрицательных значений столбца days_employed

format(data[data['days_employed'] < 0]['days_employed'].count() / data['days_employed'].count(), '.2%')

'82.20%'

В столбце **days_employed** 82,2% данных имеют отрицательное значение, предположительно, это техническая ошибка и необходимо будет заменить "-" на "+". По величинам можно сделать предварительный вывод о том, что информация представлена в днях, однако уже при рассмотрении первых строк таблицы заметны аномально большие значения, которые представлены в часах. Необходимо будет перевести их в дни.

In [4]:
# просмотр уникальных значений столбца debt

data['debt'].unique()

array([0, 1])

Столбец **debt** имеет 2 типа значений типа int, **1** - клиент имел просрочки по оплате кредита, **0** - клиент не имел просрочек в оплатах.

In [5]:
# просмотр уникальных значений столбцов education_id и education

print(data['education_id'].unique())
print(data['education'].unique())

[0 1 2 3 4]
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


В банке разделяют потребителей на 5 типов по уровням образования, далее необходимо будет привести значения столбца **education** к единому регистру и соотнести с идентификаторами.

In [6]:
# просмотр уникальных значений столбцов family_status_id и family_status

print(data['family_status_id'].unique())
print(data['family_status'].unique())

[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


Здесь также используется разделение на 5 типов по семейному статусу, но теперь количество уникальных значений соответствует количеству идентификаторов.

In [7]:
# просмотр и подсчет уникальных значений столбца children

data.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Мы видим, что есть 47 строк со значением -1 ребенок, что предположительно может означать отсутствие детей. Значение 20 детей, указанное в целых 76 строках, также считаю аномальным, так как это 0,3% от общего числа строк. Здесь могла быть опечатка, так как клавиши 2 и 0 находятся рядом на панеле numpad клавиатуры, в указанных строках целесообразно будет заменить значение 20 на 2

In [8]:
# просмотр значений в столбце purpose

data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

По значениям в столбце цели получения кредита, можно сделать вывод, что необходима лемматизация данных, так как цели покупки можно сгруппировать по аналогии и привести к единому виду (например, "сыграть свадьбу" и "на проведение свадьбы" можно считать за одну причину). Выполним лемматизацию данного столбца в процессе предобработки данных.

**Вывод** 

В каждой строке таблицы - данные о конкретном кредитном клиенте банка. Колонка **debt** будет служить основой для сопоставления, в ней указано наличие/отсутствие просроченной задолженности по кредиту.
В других колонках представлена информация о клиенте - цели кредита, наличие детей, семейное положение, доход, рабочий стаж, возраст, образование. 
В данных по столбцам трудовой стаж и ежемесячный доход имеются пропуски, их нужно будет заполнить, так как они составляют 10% от всех данных, что является значительной долей. Заполнение значений по столбцу **total_income** можно будет сделать на основе медианных или средних значений по категориям, выделенным по столбцу **income_type**.
Изучив значения столбцов, можно сделать вывод, что необходима лемматизация данных столбца **purpose** и приведение к единому регистру данных в столбце **education**.

Приступим к устранению проблем в данных.
___

## Предобработка данных

### Обработка пропусков

Посчитаем количество пропусков в таблице

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
# проверка связи в пропусках между days_employed и total_income

data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)].shape[0]

2174

Во всех строках с пропущенными значениями отсутствуют данные одновременно и о рабочем стаже, и о доходе.
Перед тем, как заполнять пропущенные значения, приведем имеющиеся к общему виду - ***стаж в днях***.
Причиной разницы в представленных данных столбца **days_employed** могут быть разные источники выгрузки данных, в одном из них данные получены в днях и по технической ошибке со знаком минус, а в другом данные были собраны в часах.

Для целей исследования пропуски в столбцах **days_employed** и **total_income** являются критичными, так как один из вопросов исследования - зависимость уровня дохода и возврата кредита в срок. Для того, чтобы оценить средние и медианные значения уровня дохода и заполнить пропуски, необходимо разделить на категории клиентов банка и одним из факторов для категоризации будет служить рабочий стаж.

Прежде чем отработать пропуски в столбце **days_employed**, которые предположительно имеют зависимость от типа занятости, проверим значения в столбце **dob_years**.

In [11]:
# проверка значений в столбце dob_years

data.groupby('dob_years')['dob_years'].count()

dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

**101** человек имеет возраст **0**. Заполним возраст средними значениями с группировкой людей по типу занятости.

In [12]:
# заполнение нулей в столбце dob_years
print(data['income_type'].unique())
print('Нулевых значений возраста до:', data[data['dob_years'] == 0]['dob_years'].count())
for income_type in data['income_type'].unique():
    mean = round(data.loc[(data['income_type'] == income_type) & (data['dob_years'] != 0), 'dob_years'].mean())
    print(income_type, mean)
    data.loc[(data['income_type'] == income_type) & (data['dob_years'] == 0), 'dob_years'] = mean
print('Нулевых значений возраста после:', data[data['dob_years'] == 0]['dob_years'].count())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Нулевых значений возраста до: 101
сотрудник 40
пенсионер 59
компаньон 40
госслужащий 41
безработный 38
предприниматель 42
студент 22
в декрете 39
Нулевых значений возраста после: 0


Нулевые значения в столбце **dob_years** заполнены, перейдем к обработке пропусков в столбце **days_employed**:

In [13]:
# определение минимального положительного рабочего стажа в таблице

print(f"Минимальный рабочий стаж в часах: {data[data['days_employed'] >= 0]['days_employed'].min()}")
print(f"Максимальный рабочий стаж в днях: {abs(data[data['days_employed'] < 0]['days_employed'].min())}")

Минимальный рабочий стаж в часах: 328728.72060451825
Максимальный рабочий стаж в днях: 18388.949900568383


Минимальный рабочий стаж в часах по данным столбца **days_employed** равен 328728,72. Переведем данные со значением более 300 000,00 в дни, а отрицательные значения, представленные в днях, преобразуем в положительные (по модулю), за границу примем 20 000,00

In [14]:
# 1. преобразование отрицательных значений в положительные
# 2. преобразование значений в часах в дни

data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] > 300000, 'days_employed'] = data.loc[data['days_employed'] > 300000, 'days_employed'] / 24
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Теперь столбец **days_employed** содержит рабочий стаж в днях, далее заполним пропуски медианными значениями по типу занятости (**income_type**)

In [15]:
# вычисление медианных значений стажа по типу занятости

print('Пропуски до:', data['days_employed'].isna().sum())

for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'days_employed'].median()
    print(income_type, '-', median)
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == income_type), 'days_employed'] = median

Пропуски до: 2174
сотрудник - 1574.2028211070851
пенсионер - 15217.221094405466
компаньон - 1547.3822226779334
госслужащий - 2689.3683533043886
безработный - 15267.235531008522
предприниматель - 520.8480834953765
студент - 578.7515535382181
в декрете - 3296.7599620220594


In [16]:
# проверка пропусков в столбце days_employed

print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски после: 0


Пропуски в столбце стаж (**days_employed**) заполнены.

Следующий шаг - устраним аномалии в столбце **children**, выявленные ранее, а именно скорректируем количество детей 20 на 2 и значение -1 на 0, так как можно предположить, что при сборе данных -1 означало отсутствие детей.

In [17]:
# устранение аномалий столбца children
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 0

Проверим значения столбца **children** после обработки:

In [18]:
# просмотр и подсчет уникальных значений столбца children после обработки

data.groupby('children')['children'].count()

children
0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Информация по количеству детей стала более реалистичной, отсутствуют отрицательные значения детей и 76 избыточно многодетных семей стали обладателями двух детей, вместо 20.

Следующий шаг - обработка пропусков в столбце **total_income**, проверим разброс значений для определения более корректного показателя для заполнения:

In [19]:
print('Максимальное значение дохода:', data['total_income'].max())
print('Минимальное значение дохода:', data['total_income'].min())

Максимальное значение дохода: 2265604.028722744
Минимальное значение дохода: 20667.26379327158


В связи с большой разницей между минимальным и максимальным доходами в выборке, более корректным будет использование медианы для заполнения пропусков. Заполним пропуски в столбце **total_income** медианными значениями с группировкой по типу занятости.

In [20]:
# заполнение пропусков медианными значениями по типам занятости: вывод пропусков, заполнение пустых значений, проверка результата

print('Пропуски до', data['total_income'].isna().sum())

for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, median)
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income_type), 'total_income'] = median

print('Пропуски после', data['total_income'].isna().sum())

Пропуски до 2174
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Пропуски после 0


In [21]:
# проверка пропусков по всем столбцам датасета
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски обработаны, резюмируем:

Аномальные значения столбца **возраст** - заменены с 0 на средние значения по типам занятости;

Аномальные значения столбца **количество детей** - заменены с 20 на 2 и с -1 на 0 (20 детей считаем аномальным значением, так как людей с таким количеством детей в первоначальном датасете более 70, значение -1 считаем аномальным, так как оно предположительно обозначало в источнике из которого получены данные отсутствие детей);

Пропуски в столбце **рабочий стаж** - заполнены медианными значениями по типам занятости;

Пропуски в столбце **ежемесячный доход** - заполнены медианными значениями по типам занятости.



### Замена типа данных

Следующий шаг обработки данных - изменение некорректных типов данных.

В исходном датасете есть 2 столбца с вещественными значениями: **days_employed** и **total_income**.

Так как дни правильнее исчислять целыми числами, то приведем столбцец **days_employed** к целочисленным значениям. Изменение типа данных осуществляется методом .astype(), так как он позволяет перевести в нужный тип.

Ежемесячный доход, судя по размерности значений, представлен в рублях, либо в иной валюте, что предполагает два знака после запятой. Также приведем к нормальному виду при помощи функции round(), округляющей значения до определенного аргументом количества знаков после запятой.

Еще раз напомним типы данных в таблице:

In [22]:
# отображение типов данных по столбцам
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [23]:
# изменение вещественных значений столбца "стаж" на целочисленные

data['days_employed'] = data['days_employed'].astype(int)
data['days_employed'].head(3)

0    8437
1    4024
2    5623
Name: days_employed, dtype: int64

In [24]:
# округление столбца "ежемесячный доход" до сотых

data['total_income'] = round(data['total_income'], 2)
data['total_income'].head(3)

0    253875.64
1    112080.01
2    145885.95
Name: total_income, dtype: float64

**Вывод**

Теперь данные соответствуют смыслу столбцов и подготовлены для дальнейших шагов предобработки.

In [25]:
# просмотр первых строк после замены типа и разрядности у вещественных переменных датасета

data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


### Обработка дубликатов

Следующий шаг - обработка дубликатов.

Перед использованием встроенных методов для подсчета количества дубликатов, приведем к нижнему регистру все символы в значениях столбца **education**. Ранее мы рассмотрели уникальные значения данного столбца и увидели повторяющиеся значения, но в разным регистрах. Появление дубликатов значений в данном столбце может быть связано с подгрузкой информации в датасет из разных источников, в одном из них данные записывались в верхнем регистре, в другом - в нижнем.

In [26]:
# приведение значений столбца education к нижнему регистру

data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Количество уникальных значений столбца **education** соответствует столбцу с id значений образования.

Найдем явные дубликаты при помощи метода duplicated() в связке с методом sum() - это позволит определить количество дублированных строк.

In [27]:
# подсчет дубликатов

print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 71


Мы нашли 71 дубликат в датасете, относительно количества строк это не большое значение, возможна техническая ошибка и данные задублировались. Удалим явные дубликаты методом drop_duplicates() и применим метод reset_index() для выравнивания индексов в датасете

In [28]:
# удаляем дубликаты и проверяем еще раз датасет

data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов:', data.duplicated().sum())

Количество дубликатов: 0


Ранее мы рассматривали уникальные значения столбца **family_status** - дубликатов значений в нем не обнаружено, тоже самое можно сказать о столбце **income_type**. Проверим столбец **gender**, если вдруг в разных базах данных использовались нижние и верхние регистры, либо кирилические обозначения пола:

In [29]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Столбец "пол клиента" имеет всего 3 значения, ошибок и повторяющихся значений в нем нет, изменение данных не требуется.

Выведем первые строки датасета:

In [30]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


**Вывод**

Явные дубликаты удалены, значения столбца **education** приведены в единый регистр.

Для подготовки датасета к анализу данных осталось провести лемматизацию столбца **purpose**.

### Лемматизация

Еще раз рассмотрим значения столбца **purpose**:

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Мы видим, что все причины взятия кредита приведены к нижнему регистру, значит использовать методы для приведения к нижнему регистру не нужно. Так как в списке есть аналогичные по смыслу значения, такие как "приобретение автомобиля", "на покупку своего автомобиля", "на покупку поддержанного автомобиля", то для качественного анализа и категоризации данных оптимальным вариантом удаления дублирующихся причин взятия кредита будет **лемматизация**.

Воспользуемся библиотекой pymystem3

In [32]:
# импорт библиотеки

from pymystem3 import Mystem
m = Mystem()

purposes = data['purpose'].unique()
lemmas = m.lemmatize(' '.join(purposes))
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Мы получили список слов, используемых в целях взятия кредита, в начальной форме. Для замены дубликатов целей получения кредита посчитаем наиболее часто встречаемые слова и на их основе дальше сформируем набор релевантных и более конкретизированных целей для упрощения дальнейшего анализа данных.

Для подсчета используем контейнер Counter из модуля collections.

In [33]:
# подсчет частоты уникальных лемматизированных слов столбца purpose

from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


По частоте лемматизированных слов определим основные цели взятия кредита.

1. "Недвижимость", "жилье", "строительство", "ремонт" - определим в одну категорию **недвижимость**, цели в данной категории могут быть ремонт дома, покупка жилья, покупка недвижимости и т.п.

2. "Автомобиль" выделим в отдельную категорию с таким же названием - **автомобиль**

3. "Свадьба" - также выделим в отдельную категорию **свадьба**, к ней относятся фразы сыграть свадьбу, проведение свадьбы

4. "Образование" часто встречаемое слово, выделим цели на дополнительное образование и получение образования в категорию **образование**

Таким образом, мы выделили 4 основные цели кредита и можем привести столбец **purpose** к более удобному виду для анализа данных и проверки гипотез.

Напишем функцию, приводящую цели взятия кредита к общему виду.

In [34]:
# функция изменения целей для столбца purpose в соответствии с новыми типами целей

def lemmas_purpose(row):
    purpose = row['purpose']
    purpose_lemmas = m.lemmatize(purpose) 
    if ('недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas):
        return 'недвижимость'
    elif 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    return 'unknown'

Применим написанную функцию при помощи метода .apply() к столбцу **purpose**

In [35]:
# создание столбца purpose_edited с измененными значениями целей

data['purpose_edited'] = data.apply(lemmas_purpose, axis=1)

# проверка на наличие значений, не прошедших лемматизацию и вернувших unknown

data[data['purpose_edited'] == 'unknown']['purpose_edited'].count()

0

**Вывод**

Для удаления неявных дубликатов мы применили лемматизацию. К таблице добавлен толбец **purpose_edited**, в котором цели получения кредита поделены на 4 категории - недвижимость, свадьба, автомобиль, образование.

Информация о целях взятия кредита приведена к виду, соответствующему для анализа данных.
Удалим столбец **purpose** и перейдем к категоризации данных, чтобы объединить категориальные переменные в группы для ответа на вопросы исследования.

In [36]:
# удаление лишнего столбца

data_new = data.drop('purpose', 1)

#первые строки предобработанного датасета

data_new.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_edited
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,недвижимость


### Категоризация данных

В исследовании необходимо ответить на 4 вопроса:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Возврат кредита в срок имеет 2 варианта - "1" (означает, что имеется задолженность), "0" (означает, что задолженностей не было)

Цели кредита сгруппированы и готовы следующему шагу исследования, наличие детей означает, что либо они есть, либо отсутствуют, поэтому данную характеристику субъектов можно категоризировать на наличие (1 и более детей) и отсутствие детей (0 детей).

Перейдем к категоризации по признаку наличия детей и затем рассмотрим данные о семейном положении и уровне дохода.

Применим изменение значений столбца **children** для определния двух категорий, где 0 - означает отсутствие детей и 1 - означает их наличие.

In [37]:
data_new['children'] = (data_new['children'] > 0).astype(int)
data_new.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_edited
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость
3,1,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба


Образование клиентов и семейное положение уже имеют категории и в исходном датасете, даже имеются id разных статусов образования и семейного положения для более удобного обращения к датасету.

Проверим соответствие классификаций присвоенным категориям id.

In [38]:
# исследуем уникальные значения столбцов family_status и family_status_id и их соответствие

data_new[['family_status', 'family_status_id']].drop_duplicates()

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


Таким образом, семейный статус *женат/замужем* соответствует индексу 0 в справочнике, *гражданский брак* - 1, *вдовец/вдова* - 2, *в разводе* - 3, *не женат/не замужем* - 4

Также проверим соответствие по образованию и справочнику с id образования

In [39]:
# исследуем уникальные значения столбцов education и education_id и их соответствие

data_new[['education', 'education_id']].drop_duplicates()

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2962,ученая степень,4


Из полученной таблицы делаем вывод, *высшее* соответствует id 0, *среднее* - 1, *неоконченное высшее* - 2, *начальное* - 3, *ученая степень* - 4

Для анализа и ответа на вопросы о влиянии доходов на возврат кредита в срок необходимо категоризировать людей по уровню дохода, чтобы обобщить единичные значения столбца **total_income**.

Еще раз рассмотрим диапазон значений столбца **total_income** при помощи метода describe.

In [40]:
data_new['total_income'].describe().apply(lambda x: format(x, 'f'))

count      21454.000000
mean      165320.081048
std        98187.301131
min        20667.260000
25%       107623.857500
50%       142594.400000
75%       195820.925000
max      2265604.030000
Name: total_income, dtype: object

Минимальный доход 20 тысяч, макимальный 2,266 млн. Слишком большая вариативность, поэтому применим децилирование, чтобы сохранить репрезентативность информации и не обобщать полученные выводы. В итоге получим категориальный ряд, поделенный на 10 отрезков и добавим для него id-обозначения категорий **от 1 до 10, где 1 - группа клиентов с самым низким ежемесячным доходом, 10 - группа клиентов с самым высоким ежемесячным доходом**

Для категоризации воспользуемся методом qcut.

In [41]:
# категоризация столбца total_income на 10 отрезков
data_new['total_income_deciled'] = pd.qcut(data_new['total_income'], q=10, precision=2)
data_new['total_income_deciled_id'] = pd.qcut(data_new['total_income'], q=10, precision=2, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Рассмотрим полученные категории и количество клиентов банка в каждом из них

In [42]:
data_new.groupby('total_income_deciled')['total_income_deciled'].count()

total_income_deciled
(20667.25, 78721.56]       2146
(78721.56, 98538.27]       2145
(98538.27, 116008.9]       2145
(116008.9, 132135.05]      2146
(132135.05, 142594.4]      2261
(142594.4, 161335.7]       2029
(161335.7, 179804.98]      2146
(179804.98, 214618.47]     2145
(214618.47, 269825.71]     2145
(269825.71, 2265604.03]    2146
Name: total_income_deciled, dtype: int64

Мы получили разделение клиентов по данным столбца **total_income**, состоящее из 10 групп по возврастаю уровня ежемесячного дохода.

Возраст напрямую не будет использоваться в поиске ответов на поставленные вопросы, однако проведем категоризацию данных столбца **dob_years**, так как возможно нам потребуется исследовать влияние возраста на другие факторы при формулировании гипотез. Используем аналогичный метод, но разделим клиентов на 5 возрастных групп.

In [43]:
# категоризация столбца dob_years на 5 отрезков
data_new['dob_years_quanted'] = pd.qcut(data_new['dob_years'], q=5, precision=0)
data_new['dob_years_quanted'].value_counts()

(18.0, 32.0]    4785
(39.0, 47.0]    4391
(47.0, 56.0]    4348
(32.0, 39.0]    4057
(56.0, 75.0]    3873
Name: dob_years_quanted, dtype: int64

Возрастные группы определены, обозначим условно категории:
- 18-32 года (юноши\девушки)
- 33-39 лет (молодые люди)
- 40-47 лет (мужчины\женщины)
- 48-56 лет (зрелые люди)
- 56-75 лет (пожилые люди)

Добавим столбец с обозначением категорий **dob_years_quanted**:

In [44]:
# id отрезков по возрасту
dob_years_quanted_id = ['юноши\девушки','молодые люди','мужчины\женщины','зрелые люди','пожилые люди']

data_new['dob_years_quanted_id'] = pd.qcut(data_new['dob_years'], q=5, precision=0, labels=dob_years_quanted_id)

Взглянем на датасет, подготовленный для поиска ответов на вопросы исследования

In [45]:
data_new.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_edited,total_income_deciled,total_income_deciled_id,dob_years_quanted,dob_years_quanted_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость,"(214618.47, 269825.71]",9,"(39.0, 47.0]",мужчины\женщины
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль,"(98538.27, 116008.9]",3,"(32.0, 39.0]",молодые люди
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость,"(142594.4, 161335.7]",6,"(32.0, 39.0]",молодые люди
3,1,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование,"(214618.47, 269825.71]",9,"(18.0, 32.0]",юноши\девушки
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба,"(142594.4, 161335.7]",6,"(47.0, 56.0]",зрелые люди
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,недвижимость,"(214618.47, 269825.71]",9,"(18.0, 32.0]",юноши\девушки
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,недвижимость,"(214618.47, 269825.71]",9,"(39.0, 47.0]",мужчины\женщины
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,"(132135.05, 142594.4]",5,"(47.0, 56.0]",зрелые люди
8,1,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,свадьба,"(78721.56, 98538.27]",2,"(32.0, 39.0]",молодые люди
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,недвижимость,"(142594.4, 161335.7]",6,"(39.0, 47.0]",мужчины\женщины


**Вывод**

В процессе категоризации данных исходного датасета мы определили категории клиентов банка для выявления зависимостей между возвратом кредита в срок и других характеристик клиентов.

Определены категории клиентов по наличию детей: **есть дети** и **нет детей**

Определены категории по семейному положению: **женат / замужем, гражданский брак, вдовец / вдова, в разводе, не женат / не замужем**

Определны категории по целям взятия кредита: **недвижимость, образование, свадьба**

Определены категории по уровню ежемесячного дохода с помощью деления на децили: **от 1 до 10, где 1 - наименьший диапазон ежемесячного дохода, 10 - наибольший диапазон ежемесячного дохода**

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для поиска ответа на заданный вопрос создадим таблицу с исследуемыми показателями: наличие детей, возврат кредита в срок.

In [46]:
# группировка по наличию детей и подсчет процента невозвратных в срок кредитов по категориям

report_one = data_new.groupby('children').agg({'debt':['count', 'sum']})
report_one['%'] = round(report_one['debt']['sum'] / report_one['debt']['count'] * 100, 2)
report_one

debt           %
          count   sum      
children                   
0         14138  1064  7.53
1          7316   677  9.25

**Вывод**

Процент людей, имеющих задолженность по возврату кредита с детьми, больше на 1,72%.

Можно сделать вывод, что при наличии детей вероятность задолженности по кредиту растет.

Возможно это связано с дополнительными обязательствами, возникающими при наличии детей, дети - это дополнительные расходы и значительная часть трат семейного бюджета, а в случае разведенных семей - это выплата алиментов. Логично предположить, что обеспечение детей у клиентов банка является более приоритетной задачей, чем выплата кредита вовремя.

Также, банки учитывают при выдаче кредита сумму содержания ребенка и для оценки платежеспособности будет уместным вычитать необходимую сумму из ежемесячного дохода

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
# группировка по семейному положению и подсчет процента невозвратных в срок кредитов в каждой группе

report_two = data_new.groupby('family_status').agg({'debt':['count', 'sum']})
report_two['%'] = round(report_two['debt']['sum'] / report_two['debt']['count'] * 100, 2)
report_two

debt          %
                       count  sum      
family_status                          
Не женат / не замужем   2810  274  9.75
в разводе               1195   85  7.11
вдовец / вдова           959   63  6.57
гражданский брак        4151  388  9.35
женат / замужем        12339  931  7.55

По расчету мы видим, что наименьшая доля задолженности по возврату кредитов у категорий вдовцы/вдовы и в разводе. Рассмотрим данный фактор в связке с возрастной группой

In [48]:
# создание таблицы для определения количества вдовцов/вдов в разных возрастных группах

data_pivot = data_new.pivot_table(index = ['dob_years_quanted_id'], columns = 'family_status', values = 'debt', aggfunc = 'count')
data_pivot

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
dob_years_quanted_id,,,,,
юноши\девушки,1173,157,11,1016,2428
молодые люди,445,200,32,835,2545
мужчины\женщины,440,298,94,905,2654
зрелые люди,386,304,263,772,2623
пожилые люди,366,236,559,623,2089


**Вывод**

Среди неженатых людей и людей в гражданском браке самая высокая доля имеющих задолженность по возврату кредита в срок, что можно объяснить более низким уровнем ответственности за свое финансовое положение.
В то же время, самый низкий процент людей с задолженностями по возврату кредита в срок наблюдается у таких категорий людей по семейному статусу, как вдовы/вдовцы, что может быть причиной серьезного жизненного опыта и понимания цены потери. Также, по количеству людей из различных возрастных групп в каждом из семейных статусов, можно сделать вывод, что количество вдовцов и вдов максимальное в возрастной группе "пожилые люди".

Соответственно неженатые и люди в гражданском браке преобладают по количеству в возрастной группе юноши/девушки (от 18 до 32 лет). Предположительно, у людей данного возраста самое нестабильное финансовое положение, и поэтому их количество влияет на результат исследования зависимости между семейным положением и возвратом кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
# группировка по уровню заработка и подсчет процента невозвратных в срок кредитов в каждой группе

report_three = data_new.groupby('total_income_deciled').agg({'debt':['count', 'sum', 'mean']})
report_three

debt               
                        count  sum      mean
total_income_deciled                        
(20667.25, 78721.56]     2146  157  0.073159
(78721.56, 98538.27]     2145  187  0.087179
(98538.27, 116008.9]     2145  179  0.083450
(116008.9, 132135.05]    2146  182  0.084809
(132135.05, 142594.4]    2261  205  0.090668
(142594.4, 161335.7]     2029  170  0.083785
(161335.7, 179804.98]    2146  181  0.084343
(179804.98, 214618.47]   2145  180  0.083916
(214618.47, 269825.71]   2145  149  0.069464
(269825.71, 2265604.03]  2146  151  0.070363

**Вывод**

По представленным данным можно сделать вывод, что процент клиентов с задолженностью по возврату кредита в срок среди людей с высоким уровнем дохода (9 и 10 категории клиентов по уровню дохода) Значит, высокий уровень дохода влияет на способность клиента вернуть кредит в срок, однако наименее обеспеченные клиенты также имеют низкий показатель просроченных кредитов, что, скорее всего, зависит от других факторов, так как людям с меньшим доходом сложнее получить кредит. 

При этом, самая большая доля клиентов с задолженностью среди самой многочисленной группы - люди со средним уровнем дохода. Им проще получить кредит, чем людям с меньшим уровнем ежемесячного дохода, но по другим факторам они могут быть хуже, если для банка ежемесячный доход имеет наибольший вес в фактороном анализе.

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
# группировка по цели кредита и подсчет процента невозвратных в срок кредитов в каждой группе при помощи сводной таблицы

report_four = data_new.pivot_table(index = ['purpose_edited'], values = 'debt', aggfunc = ['sum', 'count', 'mean'])
report_four.columns = ['debt', 'total', '%']
report_four = report_four.sort_values(by=['%'], ascending=False)
report_four

,debt,total,%
purpose_edited,,,
автомобиль,403,4306,0.093590
образование,370,4013,0.092200
свадьба,186,2324,0.080034
недвижимость,782,10811,0.072334


**Вывод**

Самая низкая доля просрочек - по жилищным кредитам, что на целых 2% меньше, чем доля просрочек по автокредитам и кредитам на образование.

## Общий вывод

Мы проверили четыре гипотезы:

1. Существует зависимость между наличием детей и возвратом кредита в срок.
2. Существует зависимость между семейным положением и возвратом кредита в срок.
3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
4. Разные цели кредита по разному влияют на его возврат в срок.

Зависимость между наличием детей и возвратом кредита в срок по представленной выборке выявлена. Клиенты с детьми имеют больше просрочек по возврату кредита с детьми больше, чем у клиентов без детей.

Зависимость между семейным положением и возвратом кредита в срок также подтвердилась, можно сделать вывод, что незамужние люди и живущие в гражданском браке имеют самую высокую долю просроченных кредитов, в то же время, вдовцы имеют самый низкий процент задолженностей. Для более детального исследования можно рассмотреть средний возраст вдовцов и обосновать меньшую долю просроченных кредитов более высоким уровнем социальной ответственности при пережитых обстоятельствах.

Уровень дохода влияет на срок возврата кредита, однако это относится к наиболее обеспеченным группам клиентов (9 и 10), у которых процент просрочки самый низкий. Самый высокий процент просроченных кредитов у людей со средним уровнем дохода (категория 5), что может быть связано с высокой долей выдачи кредитов, так как финансовый фактор зачастую имеет наибольший вес в оценке платежеспособности. В то же время, низкая доля просрочек по кредитам у наименее обеспеченной группы людей (категория 1), так как, предположительно, им выдан кредит на основе положительных характеристик по другим факторам.

При проверке четвертой гипотезы мы выявили, что разные цели действительно по-разному влияют на долю просроченных кредитов. Так, наименее рискованной оказалась выдача кредитов на покупку недвижимости или жилья, хоть и по своей сути это самый "дорогой" кредит из представленных категорий. Можно сделать вывод, что при получении крупного кредита на улучшение жилищных условий люди более ответственно относятся к своим обязательствам, в то же время самая большая доля просрочек по автокредитам, возможно это обусловлено менее выгодными условиями, по сравнению с ипотечным кредитованием. Также, почти на одном уровне по невозвратным кредитам в срок находится категория людей, взявших кредит на образование, можно предположить, что образовательные кредиты берут люди, пытающиеся сменить род деятельности или получить образование и у них наименьшая кредитная платежеспособность.